In [2]:
import os

In [3]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.ChurnPrediction.constants import *
from src.ChurnPrediction.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        #Reads YAML files (config, params, schema) using read_yaml() function defined in common.py
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        #Creates directories based on the configuration's artifacts root directory.
        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        #Creates the root directory specified in the data_ingestion section of the configuration.
        create_directories([config.root_dir])
        
        #Constructs and returns a DataIngestionConfig object with the specified attributes.
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from src.ChurnPrediction import logger
from src.ChurnPrediction.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-02 23:03:43,191: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-02 23:03:43,199: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-02 23:03:43,215: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-02 23:03:43,221: INFO: common: created directory at: artifacts]
[2023-12-02 23:03:43,226: INFO: common: created directory at: artifacts/data_ingestion]


[2023-12-02 23:03:49,569: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 267998
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c8012b582fe8e379207398a88e2c1710ea2f029ffd6c5fdee454ca1d6172207"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2654:2610:65E00C:7E5ED6:656B6A77
Accept-Ranges: bytes
Date: Sat, 02 Dec 2023 17:33:50 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10232-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1701538430.686426,VS0,VE557
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 66923e723d04e06280323942e56861fb9dee9b5a
Expires: Sat, 02 Dec 2023 17:38:50 GMT
Source-Age: 0

]
